In [144]:
# imports
import requests
import json
import pandas as pd
import numpy as np
import os

FOURSQUARE_KEY = os.getenv('FOURSQUARE_KEY')

In [145]:
#import the bike station data from csv file from part1
avaiable_bike_stations = pd.read_csv('..\\data\\available_bikes.csv')

In [146]:
avaiable_bike_stations.head()

,station_id,name,lon,lat,address,num_bikes_available,num_bikes_disabled,last_reported
0,hub_2570,Athletic Centre Bike Racks,-79.915746,43.264715,"Stearn Drive, McMaster University, Hamilton",23,0,2024-03-07 23:27:49
1,hub_624,McMaster Health Sciences,-79.918248,43.260531,"CIBC, University Avenue, Westdale, Hamilton, O...",17,0,2024-03-07 23:27:49
2,hub_626,McMaster Stadium,-79.918681,43.264622,"Stearn Drive, McMaster University, Hamilton",16,0,2024-03-07 23:27:49
3,hub_583,Discovery Drive,-79.860298,43.276067,"Waterfront Trail, Hamilton",14,0,2024-03-07 23:27:49
4,hub_547,Westdale Village,-79.905088,43.262015,"25 Marion Avenue South, Hamilton",13,0,2024-03-07 23:27:49


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [147]:
#use the method from course work to get the data from the FS API
def get_venues_fs(latitude, longitude, radius, api_key, categories):
    """
    Get venues from foursquare with a specified place type and coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query
        categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
    
    Returns:
        response: response object from the requests library.
    """
    # create the URL
    url = "https://api.foursquare.com/v3/places/search"
    # create the parameters
    params = {
        "ll" : f"{latitude},{longitude}",
        "radius" : radius,
        "categories" : categories,

    }
    # create the headers
    headers = {
        "accept": "application/json",
        "Authorization": api_key
        }
    # make the request
    resp = requests.get(url=url, params=params, headers=headers)
    # return the response
    return resp


In [148]:
#test the method using the bike station data
# get the first 5 bike stations
bike_stations = avaiable_bike_stations
#iterate through the bike stations and get the venues info from the API in json format
for index, row in bike_stations.iterrows():
    # get the response
    res = get_venues_fs(latitude=row['lat'], longitude=row['lon'], radius=1000, api_key=FOURSQUARE_KEY, categories=None)
    # convert the response to json
    res_json = res.json()
    #add the response to the dataframe
    bike_stations.loc[index, 'venues'] = json.dumps(res_json)

    

In [149]:
bike_stations

,station_id,name,lon,lat,address,num_bikes_available,num_bikes_disabled,last_reported,venues
0,hub_2570,Athletic Centre Bike Racks,-79.915746,43.264715,"Stearn Drive, McMaster University, Hamilton",23,0,2024-03-07 23:27:49,"{""results"": [{""fsq_id"": ""4d21085e4c4f60fcb765e..."
1,hub_624,McMaster Health Sciences,-79.918248,43.260531,"CIBC, University Avenue, Westdale, Hamilton, O...",17,0,2024-03-07 23:27:49,"{""results"": [{""fsq_id"": ""4b6b5d23f964a520a7032..."
2,hub_626,McMaster Stadium,-79.918681,43.264622,"Stearn Drive, McMaster University, Hamilton",16,0,2024-03-07 23:27:49,"{""results"": [{""fsq_id"": ""4b6b5d23f964a520a7032..."
3,hub_583,Discovery Drive,-79.860298,43.276067,"Waterfront Trail, Hamilton",14,0,2024-03-07 23:27:49,"{""results"": [{""fsq_id"": ""4c65f27f19f3c9b69cc79..."
4,hub_547,Westdale Village,-79.905088,43.262015,"25 Marion Avenue South, Hamilton",13,0,2024-03-07 23:27:49,"{""results"": [{""fsq_id"": ""4b81833cf964a5207fab3..."
...,...,...,...,...,...,...,...,...,...
132,hub_2868,King Street Dundas,-79.953754,43.265956,"6-8 Main Street, Hamilton",1,0,2024-03-07 23:27:49,"{""results"": [{""fsq_id"": ""4bce316fb6c49c7405c19..."
133,hub_2869,Cootes Drive Dundas,-79.947660,43.265474,"11 Court Street, Hamilton",1,0,2024-03-07 23:27:49,"{""results"": [{""fsq_id"": ""4c07a5ef2e80a59388e27..."
134,hub_603,Barton at Wentworth,-79.844958,43.258932,"501 Barton Street East, Hamilton",1,0,2024-03-07 23:27:49,"{""results"": [{""fsq_id"": ""538f7f38498e51998b970..."
135,hub_2923,Dunsmure at Sherman - ERI10,-79.838888,43.249433,"44 Dunsmure Road, Hamilton",1,0,2024-03-07 23:27:49,"{""results"": [{""fsq_id"": ""4c366e7493db0f47ec1e1..."


In [150]:
# #venues is a nested json object, so we need to flatten it to be able to use it in the analysis
# #grab the first venue from the first bike station
# venues = json.loads(bike_stations['venues'][0])
# #flatten the json object
# flattened_venues = pd.json_normalize(venues['results'])
# flattened_venues.head()

#Loop through every json object in the venues column and flatten it
#add the flattened data to a new dataframe nearby_POIs
#maintain station id for reference
nearby_POIs = pd.DataFrame()
for index, row in bike_stations.iterrows():
    #get the json object
    venues = json.loads(row['venues'])
    #flatten the json object
    flattened_venues = pd.json_normalize(venues['results'])
    #concatenate the data to the nearby_POIs dataframe
    #maintain the station id for reference
    flattened_venues['station_id'] = row['station_id']
    nearby_POIs = pd.concat([nearby_POIs, flattened_venues], ignore_index=True)


In [151]:
#from the nearby_POIs df extract the name data for the POI_category from the categories column
#flatten the categories column
#maintain the station id for reference
categories = pd.json_normalize(nearby_POIs['categories'])
categories['station_id'] = nearby_POIs['station_id']
categories

,0,1,2,3,station_id
0,"{'id': 13035, 'name': 'Coffee Shop', 'short_na...",None,None,None,hub_2570
1,"{'id': 13018, 'name': 'Pub', 'short_name': 'Pu...","{'id': 13031, 'name': 'Burger Joint', 'short_n...",None,None,hub_2570
2,"{'id': 13033, 'name': 'Bubble Tea Shop', 'shor...",None,None,None,hub_2570
3,None,None,None,None,hub_2570
4,"{'id': 10028, 'name': 'Art Museum', 'short_nam...",None,None,None,hub_2570
...,...,...,...,...,...
1360,"{'id': 10027, 'name': 'Museum', 'short_name': ...",None,None,None,hub_903
1361,"{'id': 13305, 'name': 'Burrito Restaurant', 's...",None,None,None,hub_903
1362,"{'id': 13033, 'name': 'Bubble Tea Shop', 'shor...","{'id': 13036, 'name': 'Tea Room', 'short_name'...",None,None,hub_903
1363,"{'id': 13034, 'name': 'Café', 'short_name': 'C...","{'id': 13035, 'name': 'Coffee Shop', 'short_na...",None,None,hub_903


In [152]:
#from the categories df extract the name data for the POI_category from the 0 column
#flatten the all columns except the station_id
#maintain the station id for reference
POI_category = pd.json_normalize(categories[0])
POI_category['station_id'] = categories['station_id']
#drop short_name, pluralname, id, icon.prefix, icon.suffix
POI_category = POI_category.drop(columns=['short_name', 'plural_name', 'id', 'icon.prefix', 'icon.suffix'])
POI_category

,name,station_id
0,Coffee Shop,hub_2570
1,Pub,hub_2570
2,Bubble Tea Shop,hub_2570
3,NaN,hub_2570
4,Art Museum,hub_2570
...,...,...
1360,Museum,hub_903
1361,Burrito Restaurant,hub_903
1362,Bubble Tea Shop,hub_903
1363,Café,hub_903


In [153]:
#replace the categories column in the nearby_POIs df with the POI_category df
nearby_POIs['categories'] = POI_category['name']
nearby_POIs.head()

,fsq_id,categories,chains,closed_bucket,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,...,related_places.parent.categories,related_places.parent.name,location.address,station_id,location.address_extended,related_places.children,geocodes.drop_off.latitude,geocodes.drop_off.longitude,geocodes.front_door.latitude,geocodes.front_door.longitude
0,4d21085e4c4f60fcb765e4dc,Coffee Shop,[],Unsure,195,/v3/places/4d21085e4c4f60fcb765e4dc,Union Market,America/Toronto,43.263669,-79.917443,...,"[{'id': 12013, 'name': 'College and University...",McMaster University,NaN,hub_2570,NaN,NaN,NaN,NaN,NaN,NaN
1,4b6b5d23f964a520a7032ce3,Pub,[],LikelyOpen,481,/v3/places/4b6b5d23f964a520a7032ce3,Phoenix Bar & Grill,America/Toronto,43.262878,-79.921158,...,NaN,NaN,1280 Main St W,hub_2570,NaN,NaN,NaN,NaN,NaN,NaN
2,5d9cdcaed53c6400085801a1,Bubble Tea Shop,[],Unsure,190,/v3/places/5d9cdcaed53c6400085801a1,Chatime,America/Toronto,43.263434,-79.917589,...,"[{'id': 12013, 'name': 'College and University...",Health Sciences Centre - McMaster University,1280 Main St E,hub_2570,NaN,NaN,NaN,NaN,NaN,NaN
3,4c8f915c14fdb713493a3bc4,NaN,[],Unsure,207,/v3/places/4c8f915c14fdb713493a3bc4,Travel Cuts,America/Toronto,43.263413,-79.917863,...,NaN,NaN,Musc 109C,hub_2570,NaN,NaN,NaN,NaN,NaN,NaN
4,4d7bdad5e04d6ea85865f0d5,Art Museum,[],LikelyOpen,299,/v3/places/4d7bdad5e04d6ea85865f0d5,McMaster Museum of Art,America/Toronto,43.262663,-79.918153,...,NaN,NaN,1280 Main St E,hub_2570,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
# drop the columns that are not needed
# fsq_id, chains, link, timezone, related_places.parent.fsq_id, related_places.parent.categories, location.address_extended, 
# related_places.children, geocodes.drop_off.latitude, geocodes.drop_off.longitude
nearby_POIs = nearby_POIs.drop(columns=['fsq_id', 'chains', 'link', 'timezone', 'related_places.parent.fsq_id',
                                        'related_places.parent.categories', 'location.address_extended',
                                        'related_places.children', 'geocodes.drop_off.latitude', 'geocodes.drop_off.longitude'])

In [155]:
nearby_POIs.head()

,categories,closed_bucket,distance,name,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.country,location.cross_street,location.formatted_address,location.locality,location.postcode,location.region,related_places.parent.name,location.address,station_id,geocodes.front_door.latitude,geocodes.front_door.longitude
0,Coffee Shop,Unsure,195,Union Market,43.263669,-79.917443,43.263669,-79.917443,CA,McMaster University,"McMaster University, Hamilton ON L8S 4S4",Hamilton,L8S 4S4,ON,McMaster University,NaN,hub_2570,NaN,NaN
1,Pub,LikelyOpen,481,Phoenix Bar & Grill,43.262878,-79.921158,43.262878,-79.921158,CA,in The Refectory,"1280 Main St W (in The Refectory), Hamilton ON...",Hamilton,L8S 4L8,ON,NaN,1280 Main St W,hub_2570,NaN,NaN
2,Bubble Tea Shop,Unsure,190,Chatime,43.263434,-79.917589,43.263434,-79.917589,CA,,"1280 Main St E, Hamilton ON L8K 1A9",Hamilton,L8K 1A9,ON,Health Sciences Centre - McMaster University,1280 Main St E,hub_2570,NaN,NaN
3,NaN,Unsure,207,Travel Cuts,43.263413,-79.917863,43.263413,-79.917863,CA,NaN,"Musc 109C, Hamilton ON L8S 4S4",Hamilton,L8S 4S4,ON,NaN,Musc 109C,hub_2570,NaN,NaN
4,Art Museum,LikelyOpen,299,McMaster Museum of Art,43.262663,-79.918153,43.262663,-79.918153,CA,NaN,"1280 Main St E, Hamilton ON L8K 1A9",Hamilton,L8K 1A9,ON,NaN,1280 Main St E,hub_2570,NaN,NaN


In [156]:
#drop more unneeded columns
# geocodes.main.latitude, geocodes.main.longitude, location.address, location.city, location.country, location.cross_street, 
# related_places.parent.name, location.locality, location.postal_code, location.region, geocodes.front_door.latitude, geocodes.front_door.longitude

nearby_POIs = nearby_POIs.drop(columns=['geocodes.main.latitude', 'geocodes.main.longitude', 'location.address',
                                        'location.country', 'location.cross_street', 'related_places.parent.name', 'location.locality',
                                        'location.postcode', 'location.region', 'geocodes.front_door.latitude', 'geocodes.front_door.longitude'])


Put your parsed results into a DataFrame

In [157]:
nearby_POIs
#save the data to a csv file
nearby_POIs.to_csv('..\\data\\FS_nearby_POIs.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating